---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.7MB/s 


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [0]:
from google.colab import drive
drive.mount('/data')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /data


In [0]:
import pandas as pd
from pathlib import Path as path
file_folder = path('/data/My Drive/DS1/data')

def prettify_state(input) :
    pos = input.find('[edit]')
    if pos >= 0:
        result = input[:pos]
        # if result in statesT:
        #     result = statesT[result]
        return result
    return None
def prettify_region(input) :
    pos = input.find(' (')
    if pos >= 0:
        result = input[:pos]
        return result
    return input

def read_file():
    open_filePath = file_folder / 'university_towns.txt'
    with open(open_filePath) as file:
        content = file.readlines()
    content =  [x.strip() for x in content]

    result = []
    current_state = None
    for line in content:
        next_state = prettify_state(line)
        if next_state != None:
            current_state = next_state
            continue
        region = prettify_region(line)
        result.append([current_state , region])            
    return result

def read_gdp():
    open_filePath = file_folder / "gdplev.xls"
    gdp = pd.read_excel(open_filePath,names = ["Quarter", "GDP"],usecols = "E,F", skiprows=220)
    return gdp

def find_recession():
    gdplev = read_gdp()
    recession_start_index = None
    recession_end_index = None
    for index in range(2, len(gdplev) - 2):
        if (recession_start_index == None and (gdplev.iloc[index-2][1] > gdplev.iloc[index-1][1]) and (gdplev.iloc[index-1][1] > gdplev.iloc[index][1])):
            recession_start_index = index - 2
            continue
        if recession_start_index != None and (gdplev.iloc[index+2][1] > gdplev.iloc[index+1][1]) and (gdplev.iloc[index+1][1] > gdplev.iloc[index][1]):
            recession_end_index = index + 2
            return gdplev[recession_start_index:recession_end_index + 1]

In [0]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = read_file()
    df = pd.DataFrame(data,columns=["State", "RegionName"])
    return df
print(get_list_of_university_towns())

In [0]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdplev = find_recession()
    return gdplev.iloc[0][0]
print(get_recession_start())

2008q3


In [0]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    gdplev = find_recession()
    return gdplev.iloc[len(gdplev) - 1][0]
print(get_recession_end())

2009q4


In [0]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdplev = find_recession()
    min_gdp_index = gdplev['GDP'].idxmin()
    return gdplev.loc[min_gdp_index][0]
print(get_recession_bottom())

2009q2


In [0]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    open_file_path = file_folder / "City_Zhvi_AllHomes.csv"
    cols1 = pd.date_range('2000-01', '2016-08', freq='M').strftime('%Y-%m')
    df = pd.read_csv(open_file_path, usecols=lambda x: x in cols1)
    df.columns = pd.to_datetime(df.columns)
    df.resample('Q', axis=1).mean()
    df = df.resample('Q', axis=1).mean()
    df = df.rename(columns=lambda col: '{}q{}'.format(col.year, col.quarter))
    cols2 = ["State", "RegionName"]
    df2 = pd.read_csv(open_file_path, usecols=lambda x: x in cols2)
    df = pd.concat([df2, df], join='inner', axis=1)
    return df

print(convert_housing_data_to_quarters())

In [0]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    house = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    house["Growth"] = house.apply(lambda row: 'decline' if (row[recession_start] > row[recession_bottom]) else 'growth', axis=1)
    
    return house
print(run_ttest())